In [1]:
# Usado para ler arquivos, carregar código personalizado e ter acesso a outros recursos de sistema
import os
import sys

# Programação funcional e utilitários
import funcy as fp

# Recursos para texto
import re

# Verificação de tipos
from typing import List

# Manipulação e análise dos dados
import numpy as np
import pandas as pd

# Manipulação e análise de texto
import fasttext

# Similaridade de vetores
from sklearn.metrics.pairwise import cosine_similarity

# Recursos para visualização dos dados
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

# Carregar código personalizado disponível em ../src
sys.path.append(os.path.abspath(os.path.pardir))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.utils.experiments import set_dataset_split, extract_tokens, TAG_SPLIT_PATTERN

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

# Importante: Este notebook está em estágio inicial de desenvolvimento

TODO (ou devaneios):
 - Verificar tamanho do vocabulário usando TF-IDF 
 - Fazer avaliações de similaridade e busca usando as abordagens em teste
     - [Talvez] Uma estratégia de validação pode ser a correlação entre a página de busca + posição de produtos com a mesma busca com a ordem retornada pela estratégia de trabalho com texto. A busca que deu origem à ordem certamente tem mais recursos, e também não é possível garantir que as buscas sejam da mesma pessoa, mas é algum indício.
 - Avaliar a necessidade de usar pré-processamentos adicionais de texto para o fasttext
 - [Talvez] Reduzir tamanho dos embeddings
 - [Talvez] Avaliar embeddings do BERTimbau
 


# Análise de Estratégias para Valores Textuais

O objetivo deste notebook é analisar estratégias para se lidar com valores textuais. São 4 atributos textuais disponíveis, dos quais um possui apenas 6 valores (*category*) e os demais possuem textos mais longos, especialmente tags.

As categorias possuem duas formas difernetes de uso, conforme a etapa do projeto:
 - Supervisionada: funciona como o rótulo das instâncias, então deve ser processada com algo equivalente ao *LabelEncoder*.
 - Não Supervisionada: pode ser utilizada como feature para o modelo de intenção de busca. Nesse caso, espera-se trabalhar com uma de duas opções:
    - *One Hot Encoding*, para se ter 5 features no vetor de caracerísticas
    - Alguma variação de target encoding (e.g., leave one out, catboost ou mean target encoding). Nesse caso, o número de features adicionais dependerá do número de intenções de busca identificadas. Caso o número seja superior a 5, OHE pode ser a opção mais prática.
    
Para os 3 atributos com valores textuais mais longos, as abordagens propostas inicialmente são:
 - Usar o modelo tradicional de TF-IDF: pode ser usado em conjunto com *n-grams* para estabelecer sequência entre termos (pode ser útil para título e busca), mas parece ser mais suscetível a problemas com palavras ausentes do vocabulário. Provavelmente levará a um número maior de atributos no vetor de características, mas podem ser feitas podas ou redução de dimensionalidade.
 - Usar embeddings: se beneficia do vocabulário mais amplo a partir do qual o modelo foi criado, reduzindo o problema de palavras fora do vocabulário. A depender do algoritmo e as estratégias de criação do modelo, provavelmente já incorpora informações adicionais de contexto e relação entre as palavras. Tende a ter entre 300 e 768 atributos adicionais no vetor de cacterísticas, ainda que seja possível adotar estratégias de redução.
 

   




## Carregamento dos Dados

In [2]:
columns_to_read = ['product_id', 'query', 'search_page', 'position', 'title', 'concatenated_tags', 'category', 'creation_date']

frame = pd.read_csv(os.path.join(settings.DATA_PATH, 'interim', 'training.csv'), usecols=columns_to_read)

In [3]:
print(f'Registros: {len(frame)}.')

frame.head(10)

Registros: 32746.


,product_id,query,search_page,position,title,concatenated_tags,creation_date,category
0,11394449,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,Decoração
1,15534262,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos long drink canecas,2018-04-04 20:55:07,Papel e Cia
2,15877252,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,Bebê
3,15917108,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,Decoração
4,4336889,dia dos pais,1,37,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,2018-07-11 10:41:33,Lembrancinhas
5,7544556,arranjo de flores para mesa,1,9,Arranjo de Flores - Orquidias,mini arranjos,2016-04-22 13:34:16,Decoração
6,10869150,lembrancinha maternidade,5,18,Kit Aromarizador + sacola / Lembrancinha Maternidade,bb lembrancinhas maternidade baby lembranca maternidade bebe conforto lembrancinha maternidade,2017-10-05 00:26:02,Lembrancinhas
7,13193769,chaveiro dia dos pais,1,35,chaveiro dia dos pais,dia pais,2018-07-04 12:47:49,Lembrancinhas
8,13424151,manta personalizada,1,20,Manta para bebê personalizada de Nuvem com nome,nascimento manta baby cha bebe vestido bebe,2018-04-03 16:10:51,Bebê
9,12595651,pais,1,28,Chinelo Dia dos Pais,sandalia dia pais,2018-06-25 23:23:14,Lembrancinhas


## Word2Vec (via FastTex)

Publicação: https://arxiv.org/pdf/1802.06893.pdf

Tokenização para português: http://www.statmt.org/europarl/

Fonte: https://fasttext.cc/docs/en/crawl-vectors.html

In [4]:
ft_model = fasttext.load_model(os.path.join(settings.MODELS_PATH, 'cc.pt.300.bin'))

Caso seja necessário fazer aumentação de dados para busca ou tags, é possível recuperar as palavras e expressões mais próximas:

In [5]:
ft_model.get_nearest_neighbors('bicicletas', k=10)

[(0.812232494354248, 'bikes'),
 (0.7799195647239685, 'biciletas'),
 (0.7630064487457275, 'bicicleta'),
 (0.7022727727890015, 'e-bikes'),
 (0.6933546662330627, 'Bicicletas'),
 (0.6788247227668762, 'magrelas'),
 (0.6778799295425415, 'bicletas'),
 (0.6738682389259338, 'handbikes'),
 (0.6704075932502747, 'patinetes'),
 (0.6657806038856506, 'bike')]

A seguir, são feitos alguns experimentos de verificação direta de resultados de busca de conteúdo por similaridade. O primeiro passo é gerar os embeddings de sentenças para o atributo textual escolhido -- no caso, título.

In [6]:
search_frame = (
    frame
    .copy()
    [['title']]
    .assign(embedding=lambda f: f['title'].str.lower().apply(ft_model.get_sentence_vector))
)

Tendo uma sentença de busca, faz-se sua codificação e, para os valores disponíveis para recuperação, pode-se fazer calcular uma medida de similaridade, como o cosseno, e ordenar os registros pelos maiores valores.

In [7]:
queries = ['caneca personalizada', 
           'sling',
           'canguru',
           'luminária',
           'luminária charuto'
          ]

for query in queries:
    # Codificar query
    query_embedding = ft_model.get_sentence_vector(query.lower())
    # Calcular a similaridade entre a consulta e os títulos
    search_frame['similarity'] = cosine_similarity(np.array([query_embedding]), 
                                                np.array(search_frame['embedding'].tolist())).squeeze()
    # Recuperar valores mais semelhantes
    display(HTML(f'<h4>Consulta: <strong>{query}</strong></h4>'))
    print('Resultados mais próximos:')
    display(search_frame
     .sort_values(by='similarity', ascending=False)
     .drop_duplicates('title')
     .head(10)
    )

Resultados mais próximos:


,title,embedding,similarity
1331,Caneca personalizada,"[-0.09059346, -0.034301315, 0.0607446, -0.020065779, -0.0066683497, -0.109594256, -0.072513424, -0.06625397, 0.07723059, -0.016256474, 0.039437402...",1.000000
15281,Caneca Personalizada,"[-0.09059346, -0.034301315, 0.0607446, -0.020065779, -0.0066683497, -0.109594256, -0.072513424, -0.06625397, 0.07723059, -0.016256474, 0.039437402...",1.000000
6122,CANECA PERSONALIZADA,"[-0.09059346, -0.034301315, 0.0607446, -0.020065779, -0.0066683497, -0.109594256, -0.072513424, -0.06625397, 0.07723059, -0.016256474, 0.039437402...",1.000000
10078,Caneca Porcelana Personalizada,"[-0.07069832, 0.0046982965, 0.047493562, -0.016157009, -0.008527473, -0.09306421, -0.048104625, -0.056342825, 0.06474261, -0.010103639, 0.05388523...",0.927673
27059,Caneca Preta Personalizada,"[-0.051833153, -0.01854288, 0.07233629, 0.016803812, -0.016112, -0.10689729, -0.056004763, -0.052474853, 0.041784555, -0.035726435, 0.046542794, 0...",0.923568
12346,Caneca acrílica personalizada,"[-0.061020236, -0.018148094, 0.057960864, -0.010853151, 0.002238702, -0.08773321, -0.06855093, -0.03380088, 0.039526537, -0.005547813, 0.026883181...",0.921367
18439,Caneca Acrílica Personalizada,"[-0.061020236, -0.018148094, 0.057960864, -0.010853151, 0.002238702, -0.08773321, -0.06855093, -0.03380088, 0.039526537, -0.005547813, 0.026883181...",0.921367
22940,Caneca Acrílico Personalizada,"[-0.07212547, -0.027883325, 0.068373695, -0.006781872, -0.006588269, -0.08047559, -0.06045576, -0.020821337, 0.048221037, -0.0020321584, 0.0415500...",0.919942
15771,Caneca Térmica Personalizada,"[-0.05758532, -0.05803579, 0.05319626, -0.015243884, 0.009728219, -0.116347015, -0.06378813, -0.065472856, 0.045696612, -0.034639355, 0.04596258, ...",0.918677
31625,Caneca Acrilico Personalizada,"[-0.07470423, -0.030835252, 0.058524206, -0.0045872806, -0.008715605, -0.08522179, -0.07237088, -0.021820134, 0.061030313, 0.0034812894, 0.0516236...",0.918476


Resultados mais próximos:


,title,embedding,similarity
29573,Sling bordado canguru,"[-0.0022852533, -0.016764294, 0.047316425, -0.09254758, 0.008633211, -0.06656324, 0.0024755634, 0.02586161, 0.031612847, -0.06510758, 0.020087196,...",0.770731
2083,Wrap Sling Dry Fit (Para Carregar Bebês),"[-0.021999316, -0.07976433, 0.05642091, -0.04046484, -0.056687914, -0.017304031, 0.01674196, 0.008481959, 0.0017362259, -0.029013097, 0.03341182, ...",0.667925
15679,Wrap Sling Preto Liso,"[-0.016791565, -0.018664947, 0.06829013, -0.016645458, -0.00567169, -0.055515468, -0.0073864483, 0.025938105, -0.042988062, -0.029250674, 0.071795...",0.642037
7691,"Wrap Sling, Canguru Bebê, carregador de","[0.0022705495, -0.058408108, 0.030930353, -0.030740067, -0.02776073, -0.042319622, 0.023423327, -0.0031987976, 0.009514097, -0.03324899, 0.0101285...",0.630270
5322,Promoção Imperdível - Wrap Sling Canguru,"[-0.004674551, -0.03699492, 0.040862896, -0.032531597, -0.019630544, -0.048716754, 0.0046307053, -0.007966973, 0.001832119, -0.034110233, -0.01767...",0.608842
8132,"Wrap Sling de Algodão, Frete Grátis","[-0.0055482774, -0.06460292, 0.036452368, -0.040081266, -0.037070356, -0.05208096, -0.0067972164, 0.010406724, -0.0105856415, -0.03085742, -0.0030...",0.588861
5643,SLING EM MALHA - WRAP - CANGURU,"[0.001854015, -0.022504011, 0.041315883, -0.034231737, 0.0057458715, -0.03714917, -0.015401573, 0.01702277, 0.020516567, 0.005380968, 0.011238917,...",0.554420
7204,Cueiro,"[0.032061525, -0.07596725, -0.024388548, -0.051855795, 0.00074434804, -0.036232326, 0.08344118, -0.02711079, 0.004848071, -0.03154694, 0.051804848...",0.552350
14113,Manta Bebê,"[0.025320444, -0.043820426, 0.0188271, -0.101444334, -0.0040465463, -0.079622686, 0.025736105, 0.007664902, 0.016111504, -0.061543427, 0.065889105...",0.527003
8434,casaquinho Bebê,"[-0.0082769375, -0.037745297, 0.03368908, -0.09610407, 0.018707091, -0.03252883, 0.044451933, 0.017179593, 0.04026343, -0.073243335, 0.06018559, -...",0.521495


Resultados mais próximos:


,title,embedding,similarity
29573,Sling bordado canguru,"[-0.0022852533, -0.016764294, 0.047316425, -0.09254758, 0.008633211, -0.06656324, 0.0024755634, 0.02586161, 0.031612847, -0.06510758, 0.020087196,...",0.741662
24576,CANGURU ERGONÔMICO FASHION,"[0.03929396, 0.008816363, 0.02075677, -0.024711162, 0.007305559, -0.0816867, 0.01321266, 0.015083849, 0.050864264, -0.06546644, 0.009162754, 0.000...",0.702678
7691,"Wrap Sling, Canguru Bebê, carregador de","[0.0022705495, -0.058408108, 0.030930353, -0.030740067, -0.02776073, -0.042319622, 0.023423327, -0.0031987976, 0.009514097, -0.03324899, 0.0101285...",0.640065
3860,Promoção Imperdível - Wrap Sling Canguru,"[-0.004674551, -0.03699492, 0.040862896, -0.032531597, -0.019630544, -0.048716754, 0.0046307053, -0.007966973, 0.001832119, -0.034110233, -0.01767...",0.604368
26911,Naninha elefante,"[0.011046937, 0.0033619124, -0.00611943, -0.06598573, -0.029326309, -0.031608716, 0.002005808, 0.014707606, 0.062039495, -0.03250232, -0.022037335...",0.570347
23767,Macacão Urso,"[0.07128489, -0.03725581, 0.052576564, -0.06866731, 0.04185236, -0.039540626, 0.023113972, 0.008440241, 0.06437056, -0.07732297, -0.02167094, -0.0...",0.547082
26700,COFRINHO URSO MARINHEIRO,"[0.09024654, -0.02673993, 0.01617107, -0.054171722, 0.044357426, -0.06176017, 0.036698114, -0.061884265, 0.08415711, -0.06473112, -0.03844531, -0....",0.542638
17639,Elefante Amigurumi,"[0.031643543, -0.013961498, 0.03251092, -0.06754263, 0.014548184, -0.02479055, 0.010469096, 0.024443945, 0.077148, -0.07218521, -0.058466677, -0.0...",0.539341
31455,Elefante amigurumi,"[0.031643543, -0.013961498, 0.03251092, -0.06754263, 0.014548184, -0.02479055, 0.010469096, 0.024443945, 0.077148, -0.07218521, -0.058466677, -0.0...",0.539341
31596,Chaveiro Almofada Urso Marinheiro,"[0.047662016, -0.036453594, 0.023674523, -0.0017213705, 0.0004439468, -0.065631896, 0.02938016, -0.024863323, 0.07335093, -0.0675108, -0.006504684...",0.537172


Resultados mais próximos:


,title,embedding,similarity
18599,Luminária artesanal,"[-0.02170024, -0.010178967, 0.07418527, 0.006850457, 0.011529952, -0.082544014, 0.018527118, -0.05231946, 0.015703054, -0.05172264, 0.055290442, -...",0.814827
3758,Luminária Nuvem,"[-0.04124187, -0.02229874, 0.059932828, -0.0068386886, -0.010755985, -0.07521786, -0.0065066293, 0.0149485, 0.063705295, -0.031864654, 0.070955604...",0.806173
2461,Luminária Heineken,"[-0.029981326, -0.020523777, 0.05752523, 0.028150225, 0.0047409767, -0.031100262, -0.021203842, -0.06607415, 0.040422823, -0.022326881, 0.04575206...",0.793466
25103,Luminária Madeira Retrô,"[0.0152739845, -0.017619401, 0.060587727, 0.027594155, -0.006825417, -0.06199832, 0.00933631, -0.032177355, 0.03647194, -0.038609482, 0.06466025, ...",0.788782
5126,Luminária Decorativa Amarela,"[-0.019850098, -0.015410842, 0.048835825, 0.030448114, -0.023631206, -0.10288333, 0.026235543, -0.024905946, 0.015392657, -0.061861705, 0.04851865...",0.788538
20685,Vela Mini Luminária,"[-0.010087301, -0.021463372, 0.042717688, 0.061807968, -0.046435334, -0.0893307, 0.012179415, -0.057684623, -0.018539647, -0.052470006, 0.06252831...",0.785430
13688,Vela Decorativa com luminária,"[-0.015020618, -0.03617738, 0.04537362, 0.04620771, -0.022323249, -0.07617095, 0.037055116, -0.044229552, -0.008021592, -0.060404174, 0.067267306,...",0.766307
6337,Luminária Hulk,"[0.0119768325, -0.0020740414, 0.0013733841, -0.025508834, 0.0035497134, -0.025127746, -0.01892015, -0.012218602, 0.03289303, -0.02415745, 0.010119...",0.750434
24869,Lustre Plafon Luminária Cristal Acrílico,"[-0.02397722, -0.015149036, 0.0553679, 0.029952062, -0.0020172144, -0.050457656, 0.016639853, 0.026021166, 0.022743398, -0.029580062, 0.0702398, -...",0.739219
14367,luminária Pvc Borboleta Lilás,"[-0.03009791, -0.048180122, 0.0335456, 0.016850274, -0.013238626, -0.063065544, -0.013928434, 0.015083392, 0.020805616, -0.038255222, 0.010414635,...",0.732649


Resultados mais próximos:


,title,embedding,similarity
20685,Vela Mini Luminária,"[-0.010087301, -0.021463372, 0.042717688, 0.061807968, -0.046435334, -0.0893307, 0.012179415, -0.057684623, -0.018539647, -0.052470006, 0.06252831...",0.746456
18599,Luminária artesanal,"[-0.02170024, -0.010178967, 0.07418527, 0.006850457, 0.011529952, -0.082544014, 0.018527118, -0.05231946, 0.015703054, -0.05172264, 0.055290442, -...",0.734720
13688,Vela Decorativa com luminária,"[-0.015020618, -0.03617738, 0.04537362, 0.04620771, -0.022323249, -0.07617095, 0.037055116, -0.044229552, -0.008021592, -0.060404174, 0.067267306,...",0.713296
25103,Luminária Madeira Retrô,"[0.0152739845, -0.017619401, 0.060587727, 0.027594155, -0.006825417, -0.06199832, 0.00933631, -0.032177355, 0.03647194, -0.038609482, 0.06466025, ...",0.707806
2461,Luminária Heineken,"[-0.029981326, -0.020523777, 0.05752523, 0.028150225, 0.0047409767, -0.031100262, -0.021203842, -0.06607415, 0.040422823, -0.022326881, 0.04575206...",0.705942
21887,Luminária c/ vela led - papel,"[-0.02379595, -0.018410942, 0.04753528, 0.0492254, -0.013302616, -0.045232195, -0.0071159997, -0.013490352, -0.011233791, -0.020862317, 0.04165190...",0.699048
20645,Luminária pendente artesanal garrafa Jack Daniels,"[-0.00069459097, -0.03844168, 0.053395428, 0.016799696, 0.00046498826, -0.06792957, 0.024435494, -0.022436537, 0.03452574, -0.06244322, 0.04087120...",0.693442
5126,Luminária Decorativa Amarela,"[-0.019850098, -0.015410842, 0.048835825, 0.030448114, -0.023631206, -0.10288333, 0.026235543, -0.024905946, 0.015392657, -0.061861705, 0.04851865...",0.679269
4684,Luminária Personalizada Fusca Verde,"[-0.027775353, -0.013972213, 0.052032933, 0.00026114425, 0.010932053, -0.082307085, -0.018370904, 0.011824488, 0.0025868816, -0.048351403, 0.06040...",0.679101
24869,Lustre Plafon Luminária Cristal Acrílico,"[-0.02397722, -0.015149036, 0.0553679, 0.029952062, -0.0020172144, -0.050457656, 0.016639853, 0.026021166, 0.022743398, -0.029580062, 0.0702398, -...",0.671326


## Verificar termos fora do vocabulário

In [8]:
cut_off_period = '2018-05'
split_frame = set_dataset_split(frame, cut_off_period)

pseudo_training_frame = split_frame.loc[lambda f: f['group'] != 'test'].drop(columns=['group'])
pseudo_test_frame = split_frame.loc[lambda f: f['group'] == 'test'].drop(columns=['group'])

print('Conjuntos de dados:')
print(f' - Treinamento Completo: {len(frame)}')
print(f' - Pseudo-Treino: {len(pseudo_training_frame)} ({100 * len(pseudo_training_frame) / len(frame):.2f}%)')
print(f' - Pseudo-Teste: {len(pseudo_test_frame)} ({100 * len(pseudo_test_frame) / len(frame):.2f}%)')

training_values = set(extract_tokens(pseudo_training_frame, 'concatenated_tags', TAG_SPLIT_PATTERN))
test_values = set(extract_tokens(pseudo_test_frame, 'concatenated_tags', TAG_SPLIT_PATTERN))
all_values = training_values or test_values
oov_values = test_values - training_values

w2v_values = set(ft_model.get_words())
w2v_oov_values = oov_values - w2v_values

print('Tags por conjunto de dados:')
print(f' - Treinamento Completo: {len(all_values)}')
print(f' - Pseudo-Treino (A): {len(training_values)}')
print(f' - Pseudo-Teste (B): {len(test_values)}')
print(f' - Pseudo-Teste (B - A): {len(oov_values)}')

print(f'Tokens no word2vec (C): {len(w2v_values)}')
print(f'Tokens ausentes no word2vec (C - (B - A)): {len(w2v_oov_values)}')
del split_frame

Conjuntos de dados:
 - Treinamento Completo: 32746
 - Pseudo-Treino: 27144 (82.89%)
 - Pseudo-Teste: 5602 (17.11%)
Tags por conjunto de dados:
 - Treinamento Completo: 6727
 - Pseudo-Treino (A): 6727
 - Pseudo-Teste (B): 3132
 - Pseudo-Teste (B - A): 555
Tokens no word2vec (C): 2000000
Tokens ausentes no word2vec (C - (B - A)): 134


Pesquisar algumas das tags ausentes no vocabulário do word2vec para identificar o comportamento:

In [9]:
result_frames = []
result_titles = []

for tag in list(w2v_oov_values)[:20]:
    result_titles.append(f'<h4>Tag: <strong>{tag}</strong></h4>')
    result_frames.append(pd.DataFrame(ft_model.get_nearest_neighbors(tag, k=5),
                                      columns=['Similaridade', 'Palavra'])
                         [['Palavra', 'Similaridade']]
                        )

display_side_by_side(result_frames, result_titles, padding=50)
del result_frames, result_titles

,Palavra,Similaridade
0,bencaos,0.688886
1,abencoadoras,0.618663
2,amornas,0.614769
3,medoMentirasInevitáveis,0.600029
4,bencaosdiarias,0.595239
,Palavra,Similaridade
0,sublimatico,0.754934
1,sublimatica,0.676482
2,climaticos,0.646598
3,Sublimatico,0.634133


Pelos resultados, há expressões que devem receber tratamento adicional para remover caracteres duplicados (e.g., 'querooooooooooooooooooo') ou *tokenização* mais adequada (e.g., 'amor|bencaos'). O modelo conseguiu obter resultados semelhantes para erros de digitação (e.g., 'pofissao') ou conseguir obter palavras com mesma função no texto, como nomes pessoas. 

Considerando esses resultados, deve-se fazer tratamentos adicionais no texto, mas o modelo de embeddings parece fornecer uma boa descrição das palavras para este estudo de caso.